In [1]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [11]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

#define URL for the API request
url = 'https://api.ravelry.com/color_families.json'
#make the request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
#close the connection
r.close()

#print("response code: {}".format(r)) #tells me the response code
#print("response code Details: {}".format(r.iter_lines())) #tells me details about the response code
#print("response output:")
#print(r.text) #tells me the output
#print("response output formatted:")

###Pre-filtering based on inventory###
# Search based on specific filters
query = 'bear'
page = 9
page_size = 5
craft = 'crochet'
knit_gauge = 5
weight = 'DK'

# Define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?query={}&page={}&page_size={}&craft={}&weight={}'.format(query, page, page_size, craft, weight)
# Make request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
# Close connection
r.close()

yarndf = pd.DataFrame(columns = ['crochet_gauge', 'id', 'knit_gauge',
                                 'max_gauge', 'min_gauge', 'name',
                                 'ply', 'wpi', 'yardage', 'Project name'])

if r.status_code == 200:
    data = r.json()
    # Extract pattern ID(s) from the search results
    if 'patterns' in data and len(data['patterns']) > 0:
      for i in range(len(data['patterns'])):
          pattern_id = data['patterns'][i]['id']

          # Define URL to get pattern details
          pattern_url = f'https://api.ravelry.com/patterns/{pattern_id}.json'

          # Make the request for pattern details
          pattern_response = requests.get(pattern_url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
          pattern_response.close()

          # If patterns found, collect the necessary yarn info
          if pattern_response.status_code == 200:
              pattern_data = pattern_response.json()
              try:
                #print(json.dumps(json.loads(pattern_response.text), indent=4))
                yarnData = pattern_data['pattern']['yarn_weight']
                yarnData['yardage'] = pattern_data['pattern']['yardage_description']
                yarnData['Project name'] = pattern_data['pattern']['name']
                yarndf = pd.concat([yarndf, pd.DataFrame([yarnData])], ignore_index=True)
              except:
                print("No yarn weight recorded")

    else:
      print("No patterns found")
else:
  print("Unable to access patterns" + r.code())

print(yarndf)


  crochet_gauge  id knit_gauge max_gauge min_gauge name ply wpi  \
0          None  11         22      None      None   DK   8  11   
1          None  11         22      None      None   DK   8  11   
2          None  11         22      None      None   DK   8  11   
3          None  11         22      None      None   DK   8  11   
4          None  11         22      None      None   DK   8  11   

           yardage                       Project name  
0            yards               Teddy bear amigurumi  
1   87 - 109 yards             Smootch the Teddy Bear  
2            yards                Mr Beans Teddy Bear  
3  290 - 310 yards  Sleepy Bear With Monster Slippers  
4  110 - 130 yards                  Koala Pocket Bear  
